## Solution 1: Splitting criteria

### a) 

In [8]:
#| label: 1-a-1
x <- c(1, 2, 7, 10, 20)
y <- c(1, 1, 0.5, 10, 11)

compute_mse <- function (y) mean((y - mean(y))**2)

compute_total_mse <- function (yleft, yright) {
  num_left <- length(yleft)
  num_right <- length(yright)
  w_mse_left <- num_left / (num_left + num_right) * compute_mse(yleft)
  w_mse_right <- num_right / (num_left + num_right) * compute_mse(yright)
  w_mse_left + w_mse_right
}

split <- function(x, y) {
  # try out all unique points as potential split points and ...
  unique_sorted_x <- sort(unique(x))
  split_points <- head(unique_sorted_x, length(unique_sorted_x) - 1) + 
    0.5 * diff(unique_sorted_x)

  node_mses <- lapply(
    split_points, 
    function(i) { 
      y_left <- y[x <= i]
      y_right <- y[x > i]
      # ... compute SS in both groups
      mse_split <- compute_total_mse(y_left, y_right)
      print(sprintf("split at %.6f: empirical risk = %.2f", i, mse_split))
      mse_split
    }
  )

  # select the split point yielding the maximum impurity reduction
  split_points[which.min(node_mses)]
}

In [9]:
#| label: 1-a-2
split(x, y) # 3rd obs is best split point

[1] "split at 1.500000: empirical risk = 19.14"
[1] "split at 4.500000: empirical risk = 13.43"
[1] "split at 8.500000: empirical risk = 0.13"
[1] "split at 15.000000: empirical risk = 12.64"


[1] 8.5

In [10]:
#| label: 1-a-3
split(log(x), y) # again, 3rd obs wins

[1] "split at 0.346574: empirical risk = 19.14"
[1] "split at 1.319529: empirical risk = 13.43"
[1] "split at 2.124248: empirical risk = 0.13"
[1] "split at 2.649159: empirical risk = 12.64"


[1] 2.124248

### b)

- For regression trees, we usually identify *impurity* with *variance*. Here is why:
  - It is reasonable to define impurity via the deviation between actual target values and the predicted constant -- either using absolute or square distances to enforce symmetry of positive and negative residuals.
  - Recall the constant \(L2\) risk minimizer for a node \(\mathcal{N}\):
    $$
    \bar y = \arg\min_c \frac{1}{|\mathcal{N}|}  \sum_{i = 1}^{|\mathcal{N}|} (y_i - c)^2,
    $$
    because

    \begin{align*}
    \min_c \frac{1}{|\mathcal{N}|} \sum_{i = 1}^{|\mathcal{N}|} (y_i - c)^2 &\rightarrow \frac{\partial}{\partial c} \left( \frac{1}{|\mathcal{N}|} \sum_{i = 1}^{|\mathcal{N}|} (y_i^2 - 2y_i c + c^2) \right) = 0 \\
    &\rightarrow \frac{1}{|\mathcal{N}|} \left( \sum_{i = 1}^{|\mathcal{N}|} (-2y_i + 2c) \right) = 0 \\
    &\rightarrow \sum_{i = 1}^{|\mathcal{N}|} (-2y_i + 2c) = 0 \\
    &\rightarrow -2 \sum_{i = 1}^{|\mathcal{N}|} y_i + 2|\mathcal{N}|c = 0 \\
    \end{align*}
    
    This implies $\hat{c} = \frac{1}{|\mathcal{N}|} \sum_{i = 1}^{|\mathcal{N}|} y_i = \bar{y}$.

  - Consequently, we have 
    $$
    \bar y = \arg\min_c \frac{1}{|\mathcal{N}|}  \sum_{i = 1}^{|\mathcal{N}|} (y_i - c)^2,
    $$
    where the right hand side is the (biased) sample variance for sample mean \(c\).
  - Therefore, predicting the sample mean both minimizes risk under \(L2\) loss and variance impurity.
  - Since constant mean prediction is equivalent to an intercept LM (minimizing the sum of squared residuals!), regression trees with \(L2\) loss perform piecewise constant linear regression.
  - The same correspondence holds between impurity via absolute distances and \(L1\) regression.
